# Wine Inventory

There are a collection of wines that have been cataloged by what we have in the house "onShelf.csv" and by what we previously had but no longer have "noInventory.csv".

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# creating a single dataset of all the wines

headers = ['Location Code', 'Location', 'Vintage', 'Region', 'Wine Name', 'Grape/Blend', 'Quantity', 
           'Rating', 'Unit Purchase Price', 'Type', 'Purchase Date']

in_stock = pd.read_csv('onShelf.csv', names=headers, header=0, index_col=False)
no_stock = pd.read_csv('noInventory.csv', names=headers, header=0, index_col=False)

no_stock['Quantity'] = no_stock['Quantity'].replace([1], 0)

df = pd.concat([in_stock, no_stock])

In [4]:
df.head()

,Location Code,Location,Vintage,Region,Wine Name,Grape/Blend,Quantity,Rating,Unit Purchase Price,Type,Purchase Date
0,TL1,Bar,2017.0,France,Domaine Lafage Bastide Miraflors Côtes du Rous...,Syrah/Granache,1.0,NaN,$14.25,Red,11/7/2020
1,TL2,Bar,2017.0,Italy,Scaia Paradiso,Corvina/Rondinella/Cabernet,1.0,4.0,$19.00,Red,5/30/2020
2,TL3,Bar,2016.0,Spain,Nubori Crianza Rioja,Tempranillo,1.0,4.0,$16.00,Red,11/23/2019
3,TL3,Bar,2016.0,Spain,Cyan Toro,Tempranillo,1.0,NaN,$16.50,Red,11/7/2020
4,TL4,Bar,2016.0,California,Sterling,Merlot,1.0,4.0,$-,Red,NaN


In [5]:
df.describe()

,Vintage,Quantity,Rating
count,176.000000,181.000000,134.000000
mean,2015.948864,0.883978,4.220149
std,2.133461,1.039668,0.754712
min,2004.000000,0.000000,0.000000
25%,2015.000000,0.000000,4.000000
50%,2016.000000,1.000000,4.000000
75%,2017.000000,2.000000,5.000000
max,2019.000000,3.000000,5.000000


### Handling Missing Values

In [6]:
df.drop(df[df['Wine Name'].isna() == True].index, inplace=True)

In [7]:
df.isna().sum()

Location Code           5
Location                5
Vintage                 5
Region                  0
Wine Name               0
Grape/Blend             0
Quantity                0
Rating                 47
Unit Purchase Price     8
Type                    1
Purchase Date          75
dtype: int64

In [14]:
df['Rating'] = df['Rating'].fillna(0)
df['Vintage'] = df['Vintage'].fillna(0)

In [15]:
df = df.astype({'Quantity':'int', "Rating":'int', 'Vintage':'int'}) 

In [16]:
df

,Location Code,Location,Vintage,Region,Wine Name,Grape/Blend,Quantity,Rating,Unit Purchase Price,Type,Purchase Date
0,TL1,Bar,2017,France,Domaine Lafage Bastide Miraflors Côtes du Rous...,Syrah/Granache,1,0,$14.25,Red,11/7/2020
1,TL2,Bar,2017,Italy,Scaia Paradiso,Corvina/Rondinella/Cabernet,1,4,$19.00,Red,5/30/2020
2,TL3,Bar,2016,Spain,Nubori Crianza Rioja,Tempranillo,1,4,$16.00,Red,11/23/2019
3,TL3,Bar,2016,Spain,Cyan Toro,Tempranillo,1,0,$16.50,Red,11/7/2020
4,TL4,Bar,2016,California,Sterling,Merlot,1,4,$-,Red,NaN
...,...,...,...,...,...,...,...,...,...,...,...
83,TR2,Rack,2016,Chile,Casa Silva,Cabernet,0,5,$20.00,Red,6/30/2019
84,TR6,Rack,2017,France,Domaine Palon Cotes du Rhone,Cinsaut - Grenache - Syrah,0,4,$17.00,Red,10/12/2019
85,LB5,Wrack,2017,Uruguay,Garzon,Tannat,0,4,$19.00,Red,3/30/2019
86,TL2,Bar,2016,California,The Fabelist 067,Tempranillo,0,2,$-,Red,6/30/2019


In [28]:
df['Quantity'][68].replace([1], 0)

68    0
68    0
Name: Quantity, dtype: int32

In [36]:
df[df['Wine Name']=='Mara']

,Location Code,Location,Vintage,Region,Wine Name,Grape/Blend,Quantity,Rating,Unit Purchase Price,Type,Purchase Date
68,WF,Wfridge,2016,Italy,Mara,Valpolicella Ripasso,0,4,$24.00,Red,6/30/2019


In [37]:
df.loc[68]

,Location Code,Location,Vintage,Region,Wine Name,Grape/Blend,Quantity,Rating,Unit Purchase Price,Type,Purchase Date
68,WF,Wfridge,2016,Italy,Mara,Valpolicella Ripasso,0,4,$24.00,Red,6/30/2019
68,NaN,NaN,2018,Italy,Gio Delle Venezie,Pinot Grigio,0,0,$-,White,NaN
